In [1]:
import json
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import re
import os
import sys
import javalang
from xml.dom.minidom import parse
import xml.dom.minidom

In [ ]:
def variable_query(root, name, target_list):
    children = None
    if root is None:
        #print(root)
        return 1
    
    if isinstance(root, javalang.ast.Node):
        children = root.children
    elif isinstance(root, (list, tuple)):
        children = root
    else:
        return
    for child in children:
        if isinstance(child, (javalang.tree.VariableDeclarator)) and child.name == name:
            target_list.append(child)
        if isinstance(child, (javalang.ast.Node, list, tuple)):
            variable_query(child,name,target_list)

            
def permission_check(permission_list,file_path):
    try:
        DOMTree = xml.dom.minidom.parse(file_path)
    except:
        return False
    collection = DOMTree.documentElement
    permission_tags = collection.getElementsByTagName('uses-permission')
    for tag in permission_tags:
        name = tag.getAttribute("android:name")
        if name in permission_list:
            return False
    return True
def factors_check(name_list,text):
    for name in name_list:
        if name in text:
            return True
    return False

In [ ]:
def walk_tree_animation(root,violation_list,var_names):
    children = None
    if root is None:
        #print(root)
        return 
    
    if isinstance(root, javalang.ast.Node):
        children = root.children
    elif isinstance(root, (list, tuple)):
        children = root
    else:
        return
    
    for child in children:
        
        if isinstance(child, (javalang.tree.FieldDeclaration,javalang.tree.LocalVariableDeclaration)):
            if 'Animator' in child.type.name:
                for declarator in child.declarators:
                    if isinstance(declarator, (javalang.tree.VariableDeclarator)):
                        var_names.append(declarator.name)
        
        
        if isinstance(child, javalang.tree.This):
            for member in child.selectors:
                if isinstance(member, (javalang.tree.MethodInvocation)):
                    
                    if isinstance(child.selectors[0], javalang.tree.MemberReference):
                        flag = 0
                        method_qualifier = None
                        if child.selectors[0].member in var_names and member.member == 'setDuration':
                            for argument in member.arguments:
                                if isinstance(argument, (javalang.tree.MemberReference)):
                                    if argument.member.isnumeric():
                                        if int(argument.member) >= 2000:
                                            violation_list.append('violation')
                                    else:
                                        violation_list.append(argument.member)
                                        
                        if child.selectors[0].member in var_names and member.member == 'setRepeatCount':
                            flag = 1
                            method_qualifier = member.qualifier
                        for argument in member.arguments:
                            if isinstance(argument, (javalang.tree.MemberReference)):
                                if flag and argument.member == 'INFINITE' and argument.qualifier == 'Animation':
                                    violation_list.append('violation')
        
        if isinstance(child, (javalang.tree.MethodInvocation)):
            #print(child)
            #print('method ',child.member)
            #print('qualifier ',child.qualifier)
            flag = 0
            method_qualifier = None
            if child.qualifier in var_names and child.member == 'setDuration':
                for argument in child.arguments:
                    if isinstance(argument, (javalang.tree.MemberReference)):
                        if argument.member.isnumeric():
                            if int(argument.member) >= 2000:
                                violation_list.append('violation')
                        else:
                            violation_list.append(argument.member)
                        #print('argument ',argument.member)
            if child.qualifier in var_names and child.member == 'setRepeatCount':
                flag = 1
                method_qualifier = child.qualifier
            for argument in child.arguments:
                if isinstance(argument, (javalang.tree.MemberReference)):
                    if flag and argument.member == 'INFINITE' and argument.qualifier == 'Animation':
                        violation_list.append('violation')
                        
        if isinstance(child, (javalang.ast.Node, list, tuple)):
            walk_tree_animation(child,violation_list,var_names)


In [ ]:
def animation_hedonism_detector(file_path,text,treenode):
    if text == False and treenode == False :
        return 0
    factor_list = ['Animator', 'setDuration', 'setRepeatCount','Animation', 'INFINITE']
    if not factors_check(factor_list, text):
        return 0
    
    violation_list = list()
    name_list = list()
    walk_tree_animation(treenode,violation_list,name_list)
    if 'violation' in violation_list:
        print('animation_hedonism violation')
        return 1
    for item in violation_list:
        if not item == 'violation':
            target_list = []
            variable_query(treenode,item,target_list)
            #print(file,target_list)
            for target_var in target_list:
                try:
                    if int(target_var.initializer.value) >= 2000:
                        print('animation_hedonism violation')
                        return 1
                except:
                    pass
    return 0

In [ ]:
i=0

pridir_code = 'D:/jupyterCode/summer_object/apks/decomplied/'
for code_folder in os.listdir(pridir_code)[:]:
    #with open('text.txt','r') as file:
    #    code_folder = file.read()
    if code_folder.endswith(".zip"):
        continue
        
    row = {'apk_name':code_folder,'mtp_SD':0,'hardware_security':0,'nfc_SD':0,'animation_hedonism':0, \
           'media_SD':0,'media_hedonism':0,'media_Uni':0,'nfc_security':0,\
       'telephony_security':0,'telephony_conformity':0}
    full_code_folder = pridir_code+code_folder
    for root, dirs, files in os.walk(full_code_folder):
        for file in files:

            try:
                if file.endswith(".java"):
                    file_path = os.path.join(root, file)

                    try:
                        with open(file_path, "r") as file:
                            text = file.read()
                        treenode = javalang.parse.parse(text)
                    except:
                        text = False
                        treenode = False
                        continue
                    #animation
                    if not row['animation_hedonism']:
                        row['animation_hedonism'] = animation_hedonism_detector(file_path,text,treenode)

            except:
                print('error')
    i+=1